In [1]:
# must run as conda virtual environemnt: spatial-venv

# geopandas and GDAL / OSGEO
# import geopandas as gpd
# standard imports
import sys



In [2]:
# https://pcjericks.github.io/py-gdalogr-cookbook/vector_layers.html#get-all-layers-in-an-esri-file-geodatabase

In [3]:
# import OGR
from osgeo import ogr

gdb_path = "./raw-data/ElectricityTransmissionLines_v2.gdb"

# use OGR specific exceptions
ogr.UseExceptions()

# get the driver
driver = ogr.GetDriverByName("OpenFileGDB")

# opening the FileGDB
try:
    gdb = driver.Open(gdb_path, 0)
except Exception as e:
    print(e)
    sys.exit()

In [4]:
# list to store layers'names
featsClassNames = []

# looping the range between 0 and the number of Feature Classes ("shapefiles") in the gdb
for featsClass_idx in range(gdb.GetLayerCount()):
    # grab the Feature Class via it's index
    featsClass = gdb.GetLayerByIndex(featsClass_idx)
    # only retrieve the name of the Feature Class
    featsClassNames.append(featsClass.GetName())


print("Number of Feature Classes in gdb:",gdb.GetLayerCount())
# printing
for featsClassName in featsClassNames:
    print ("\t",featsClassName)


Number of Feature Classes in gdb: 1
	 ElectricityTransmissionLines_v2


In [5]:
# using pythonic naming
target_feats_class = gdb.GetLayerByName(featsClassNames[0])
print("Re-retrieved the feature class based on the name:",target_feats_class.GetName())

Re-retrieved the feature class based on the name: ElectricityTransmissionLines_v2


In [6]:
#[x for x in dir(target_feats_class) if not x.startswith("_")]
target_feats_class

<osgeo.ogr.Layer; proxy of <Swig Object of type 'OGRLayerShadow *' at 0x0346FFE0> >

In [7]:
# use extent to draw a circle around australia.
bbox = target_feats_class.GetExtent()
# lat,lon
tl = bbox[3],bbox[0]
br = bbox[2],bbox[1] 

In [8]:
geom_idx = target_feats_class.GetGeomType()
feature = target_feats_class.GetFeature(1)
geom = feature.geometry()

In [11]:
import json
# grabbing the 
feature_dict = json.loads(feature.ExportToJson())["properties"]
feature_dict

{'FEATURETYPE': 'Transmission Line',
 'DESCRIPTON': 'A network of wires and insulators used to connect and transport high voltage electricity from generators to large demand customers and the lower voltage electricity distribution network',
 'CLASS': 'Overhead',
 'FID': '1',
 'NAME': 'Callide B Power Station to Calvale',
 'OPERATIONALSTATUS': 'Operational',
 'CAPACITYKV': '275',
 'STATE': 'Queensland',
 'SPATIALCONFIDENCE': '5',
 'REVISED': '2012/07/10 00:00:00',
 'COMMENT': None,
 'SHAPE_Length': 0.010167099425277545}

first_feature = next(target_feats_class)
first_feature.ExportToJson()

In [ ]:
def abs_addition(l, val):
    # add or subtract the value from the list of elements.
    # increases the absolute value. e.g. -45+10=-55, 5+10=15
    return [(-1,1)[ele>=0]*(abs(ele)+val) for ele in l]
abs_addition((1,-25), 20)

In [ ]:
import numpy as np
print(tl,br)

In [ ]:
import folium
# get middle point within the bbox for display purposes
middle_point = np.mean((tl,br), axis=0)

m = folium.Map(location=middle_point, tiles="", zoom_start=5)
# create rectangle to represent the bounding box
folium.Rectangle((tl,br),color="#FFFF00").add_to(m)
# iterate over each feature within the feature class
for i, feature in enumerate(target_feats_class):
    # extract contents of feature by exporting the feature 
    #   to a json string, then parsing that json string to a dict
    geom_dict = json.loads(feature.ExportToJson())
    
    # capacity
    geom_dict['CAPACITYKV']
    
    # extract the coordinates from the dict. coordinates are within a list of length 1
    line_points_lonlat = geom_dict["geometry"]["coordinates"][0]
    
    # reverse coordinate tuple order to have latitude first. 
    line_points_latlon = [(lat,lon) for lon,lat in line_points_lonlat]
    
    # draw the black line of weight 2 onto the map.
    folium.PolyLine(line_points_latlon, 
                    weight=2,
                    color="#000000"
   ).add_to(m)

m.save("./results/map-1.html") 

## Current GOal
> goal is currently very vague: create a visualisation using data from a geodatabase.


In [ ]:

# clean close of geodatabase object
del gdb